In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from duneanalytics import DuneAnalytics
import numpy as np
import pandas as pd
import pandas_datareader.data as reader
import datetime as dt
import statsmodels.api as sm

2022-08-11 13:08:28,581 : INFO : _init_num_threads : NumExpr defaulting to 8 threads.


### Define Helper Functions

In [2]:
def calc_beta(df_ret, token='BTC', benchmark='SP500'):
    X = df_ret[benchmark]
    y = df_ret[token]
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm)
    results = model.fit()
    return results.params[benchmark]

In [3]:
def annualize_tot_ret(tot_ret, dur_years):
    return (1+tot_ret)**(1/dur_years) - 1

In [4]:
def extract_frame_from_dune_data(dune_data, date_col='day'):    
    dd = dune_data['data']['get_result_by_result_id']
    df = pd.json_normalize(dd, record_prefix='')
    df = df.loc[:, df.columns.str.startswith('data')]
    df.columns = df.columns.str.replace('data.', '', regex=False)
    df['date'] = pd.to_datetime(df[date_col].str.replace('T.*', '', regex=True))
    if date_col != 'date':
        df = df.drop(date_col, axis=1)
    df = df.set_index('date')
    # drop the last row cuz it may not always be a full day
    return df.iloc[:-1, :]

In [5]:
# get Dune Analytics login credentials
MY_USERNAME = os.environ.get('DUNE_USERNAME')
MY_PASSWORD = os.environ.get('DUNE_PASSWORD')
dune = DuneAnalytics(MY_USERNAME, MY_PASSWORD)

dune.login()
dune.fetch_auth_token()

# query daily prices for GLP and TriCrypto
glp_arbi_prices = dune.query_result(dune.query_result_id(query_id=1069389))
tricrypto_prices = dune.query_result(dune.query_result_id(query_id=1145739))
df_glp_prices = (extract_frame_from_dune_data(glp_arbi_prices, 'date')
    .rename({'price':'GLP'}, axis=1))
df_tri_prices = (extract_frame_from_dune_data(tricrypto_prices, 'date')
    .rename({'price':'TriCrypto'}, axis=1))
# TriCrypto price became available on 2021-06-09 and GLP on 2021-08-31. 
# let's cut TriCrypto's price data using 2021-08-31. This will ensure the 
# monthly returns to be calculated over the same months.
df_tri_prices = df_tri_prices.loc[df_glp_prices.index[0]:, :]

## Get Price Data from Yahoo

SP500, Reit, Tips, Bonds, Gold, Broad Commodities, BTC, and ETH 

We want to use the start date of the asset with the least amount of history as the start date of the period we want to download data for all assets. This saves time.

In [6]:
start = dt.date(2021, 9, 1) # GLP price first became available on 2021-08-31.
    # yahoo price reader downloads prices since the day before `start` (including the day before `start`) when running locally. 
    # But when running on streamlit cloud, it excludes the day before `start`, which should be the correct behavior. 
    # I guess it has to do with my timezone and local time? 
today = dt.datetime.now(tz=dt.timezone.utc)
end = dt.date(today.year, today.month, 1)
tickers_names = {
    '^GSPC': 'SP500',
    'VNQ': 'Real Estate',           
    'TIP': 'Inflation-Linked Bonds',   
    'BND': 'Nominal Bonds', 
    'GLD': 'Gold',
    '^SPGSCI': 'Broad Commodities',
    'BTC-USD':'BTC', 
    'ETH-USD':'ETH'
}
tickers = list(tickers_names.keys())

df_prices = reader.get_data_yahoo(tickers, start, end)['Adj Close'].rename(tickers_names, axis=1)
df_prices.columns.name = None

In [7]:
df_prices.head(2)

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH
Date,,,,,,,,
2021-08-31,4522.680176,105.651833,121.474174,84.502541,169.690002,527.369995,47166.687500,3433.732666
2021-09-01,4524.089844,107.344910,121.527061,84.517220,169.699997,526.090027,48847.027344,3834.828125


In [8]:
df_prices.tail(2)

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH
Date,,,,,,,,
2022-07-31,NaN,NaN,NaN,NaN,NaN,NaN,23336.896484,1681.517334
2022-08-01,4118.629883,98.160004,117.129997,77.080002,165.029999,671.690002,23314.199219,1635.195801


In [9]:
# drop the last row since end date is the first day of the current month, keeping it will result a fake current month return
df_prices = df_prices.iloc[:-1]

In [10]:
df_prices.tail(2)

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH
Date,,,,,,,,
2022-07-30,NaN,NaN,NaN,NaN,NaN,NaN,23656.207031,1695.969482
2022-07-31,NaN,NaN,NaN,NaN,NaN,NaN,23336.896484,1681.517334


In [11]:
# download risk free rates, which are already multiplied by 100, so we divide by 100
# behaves correctly starting on `start` not the day before
rfs = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0].RF / 100 
rfs.head()

Date
2021-09    0.0000
2021-10    0.0000
2021-11    0.0000
2021-12    0.0001
2022-01    0.0000
Freq: M, Name: RF, dtype: float64

## Calculate Monthly Excess Returns

In [12]:
monthly_rets = df_prices.resample('M').last().pct_change()
monthly_rets_glp = df_glp_prices.resample('M').last().pct_change()
monthly_rets_tri = df_tri_prices.resample('M').last().pct_change()
monthly_rets = monthly_rets.join(monthly_rets_glp).join(monthly_rets_tri)

In [13]:
monthly_rets.head(2)

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH,GLP,TriCrypto
Date,,,,,,,,,,
2021-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-30,-0.047569,-0.056815,-0.007801,-0.010136,-0.032235,0.057531,-0.071572,-0.125826,-0.036346,-0.067016


In [14]:
monthly_rets_glp.head(2)

,GLP
date,
2021-08-31,NaN
2021-09-30,-0.036346


In [15]:
monthly_rets_tri.head(2)

,TriCrypto
date,
2021-08-31,NaN
2021-09-30,-0.067016


In [16]:
monthly_rets = monthly_rets.to_period('M') # because the rfs have monthly period, otherwise can't join
monthly_rets = monthly_rets.join(rfs)

In [17]:
# calculate monthly excess returns
for col in monthly_rets.columns.drop('RF'):
    newcol = col + ' - ' + 'RF'
    monthly_rets[newcol] = monthly_rets[col] - monthly_rets['RF']
# ensure all assets have the same months for fair comparison.  
excess_monthly_rets = monthly_rets.dropna().loc[:, monthly_rets.columns.str.endswith('- RF')]
# remove ' - RF' from the column names for better display
excess_monthly_rets.columns = excess_monthly_rets.columns.str.replace(' - RF', '')
excess_monthly_rets.head()

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH,GLP,TriCrypto
Date,,,,,,,,,,
2021-09,-0.047569,-0.056815,-0.007801,-0.010136,-0.032235,0.057531,-0.071572,-0.125826,-0.036346,-0.067016
2021-10,0.069144,0.071330,0.011096,0.000703,0.014797,0.054688,0.400267,0.428559,0.146687,0.261740
2021-11,-0.008334,-0.021093,0.008651,0.002041,-0.006901,-0.111814,-0.070346,0.080084,0.017250,0.006004
2021-12,0.043513,0.096956,0.003929,-0.003203,0.032891,0.074052,-0.187784,-0.204969,-0.065666,-0.128538
2022-01,-0.052585,-0.084217,-0.020588,-0.020649,-0.016788,0.111551,-0.168947,-0.270012,-0.213344,-0.185582


In [18]:
excess_monthly_rets.tail()

,SP500,Real Estate,Inflation-Linked Bonds,Nominal Bonds,Gold,Broad Commodities,BTC,ETH,GLP,TriCrypto
Date,,,,,,,,,,
2022-02,-0.031360,-0.034827,0.008557,-0.011380,0.061217,0.078986,0.122394,0.085900,0.062604,0.066120
2022-03,0.035673,0.062539,-0.018855,-0.027621,0.012626,0.075882,0.054201,0.124058,0.067090,0.096874
2022-04,-0.088057,-0.040978,-0.021930,-0.039827,-0.020803,0.044861,-0.171906,-0.168143,-0.068390,-0.101642
2022-05,-0.000247,-0.047154,-0.010222,0.007997,-0.032915,0.038947,-0.157335,-0.288873,-0.099267,-0.171912
2022-06,-0.084520,-0.075089,-0.031755,-0.017220,-0.016260,-0.098788,-0.378288,-0.451105,-0.177709,-0.315428


In [19]:
print('Data period: ', excess_monthly_rets.index.min().strftime('%Y-%m'), 
      '~', excess_monthly_rets.index.max().strftime('%Y-%m'))
print("Number of months:", len(excess_monthly_rets))

Data period:  2021-09 ~ 2022-06
Number of months: 10


## Output Tables

In [20]:
# Calculate Beta, Sharpe Ratio, and Excess Return (Ann) using Excess Monthly Returns
#   - Treat SP500 as benchmark
#   - GLP and TriCrypto Yields are excluded
market = 'SP500'
tokens = excess_monthly_rets.columns
betas = [calc_beta(excess_monthly_rets, token, market).round(3) for token in tokens]
df_betas = pd.Series(betas, index=tokens).sort_values().to_frame().rename({0:'Beta'}, axis=1)

sharpe_ratios = (excess_monthly_rets.mean() / excess_monthly_rets.std()).round(3)
df_sharpes = sharpe_ratios.sort_values(ascending=False).to_frame().rename({0:'Sharpe Ratio'}, axis=1)

tot_ret = (1+excess_monthly_rets).prod()-1
dur_years = len(excess_monthly_rets) / 12
ann_excess_rets = annualize_tot_ret(tot_ret, dur_years).round(3) * 100
df_ann_excess_rets = ann_excess_rets.sort_values(ascending=False).to_frame().rename({0:'Excess Return (Ann)'}, axis=1)

In [21]:
df_sharpes.style.format(precision=3)

,Sharpe Ratio
Broad Commodities,0.431
Gold,-0.014
Real Estate,-0.198
BTC,-0.295
SP500,-0.304
ETH,-0.307
TriCrypto,-0.325
GLP,-0.326
Inflation-Linked Bonds,-0.517
Nominal Bonds,-0.808


In [22]:
df_ann_excess_rets.style.format({'Excess Return (Ann)': '{:,.1f}%'.format})

,Excess Return (Ann)
Broad Commodities,42.5%
Gold,-1.0%
Inflation-Linked Bonds,-9.2%
Nominal Bonds,-13.5%
Real Estate,-16.4%
SP500,-19.4%
GLP,-40.8%
TriCrypto,-56.4%
BTC,-64.8%
ETH,-75.4%


In [23]:
df_betas.style.format(precision=3)

,Beta
Nominal Bonds,0.146
Inflation-Linked Bonds,0.179
Gold,0.252
Broad Commodities,0.355
SP500,1.000
Real Estate,1.053
GLP,1.356
TriCrypto,2.043
BTC,2.566
ETH,3.126
